In [1]:
# Auto Reload
%load_ext autoreload
%autoreload 2

In [2]:
# Import Modules
import numpy as np
import copy
import pandas as pd
import json
import requests
from bs4 import BeautifulSoup
from pprint import pprint

In [3]:
# Import Custom Modules
from webscrape import *
from helpers import *

In [4]:
# 2019 Info
top_songs_2019 = get_top_songs(2019)

In [5]:
# 2018 Info
top_songs_2018 = get_top_songs(2018)

In [6]:
# 2017 Info
top_songs_2017 = get_top_songs(2017)

In [7]:
# 2016 Info
top_songs_2016 = get_top_songs(2016)

In [8]:
# 2015 Info
top_songs_2015 = get_top_songs(2015)

In [9]:
# Year list
years = [2015, 2016, 2017, 2018, 2019]
# List of top song tuples by year
top_songs_list = [top_songs_2015, top_songs_2016, top_songs_2017, top_songs_2018, top_songs_2019]

In [29]:
# Create DataFrame
df = pd.DataFrame(
        columns=['song', 'artist', 'featured', 'rank', 'year'],
)

In [31]:
# Populate Dataframe
for year, info in zip(years, top_songs_list):
    x = pd.DataFrame(
        data=[[*info[x], year] for x in range(len(info))],
        columns=['song', 'artist', 'featured', 'rank', 'year'])
    df = df.append(x, ignore_index=True)


In [32]:
# df.info()
df.head()

,song,artist,featured,rank,year
0,See You Again,Wiz Khalifa,,1,2015
1,Trap Queen,Fetty Wap,,2,2015
2,Watch Me,Silento,,3,2015
3,679,Fetty Wap,,4,2015
4,Hotline Bling,Drake,,5,2015


In [ ]:
responses, lyrics = [], []
for song, artist in zip(df['song'], df['artist']):
    res = ""
    try:
        response = get_extract_response(song, artist)
        if not response:
            response = get_extract_response(song, '')
        res = scrape_song_lyrics_from_url(response)
    except:
        pass
    responses.append(response)
    lyrics.append(res)

In [ ]:
# Create Series for each feature
lyric_state = pd.Series([response['result']['lyrics_state'] for response in responses]) 
song_id =  pd.Series([response['result']['id'] for response in responses])
lyric_owner_id = pd.Series([response['result']['lyric_owner_id'] for response in responses])

In [ ]:
# Create More Features for Each Song ['lyrics_state', 'id', 'lyric_owner_id']

# Match Lyrics To Song
df['lyrics'] = pd.Series(lyrics)

# Match Lyric State To Song
df['lyric_state'] = pd.Series(lyric_state)

# Match song id To Song
df['song_id'] = pd.Series(song_id)

# Match lyric owner To Song
df['lyric_owner_id'] = pd.Series(lyric_owner_id)


In [ ]:
# for x in range(len(lyrics)):
#     if not lyrics[x]:
#         print(x)
# song, artist = df.iloc[185]['song'], df.iloc[185]['artist']
# get_extract_response(song, "")

In [44]:
df

,song,artist,featured,rank,year,lyrics
0,See You Again,Wiz Khalifa,,1,2015,"It's been a long day without you, my friend\nA..."
1,Trap Queen,Fetty Wap,,2,2015,"RGF productions\nRemy Boyz, yah-ah\n1738, ayy\..."
2,Watch Me,Silento,,3,2015,"Whip, nae nae\nWhip, whip, nae nae\nWhip, nae ..."
3,679,Fetty Wap,,4,2015,"Yeaaah baby, 17\n(RemyBoy Lifestyle)\nAy, ay, ..."
4,Hotline Bling,Drake,,5,2015,"You used to call me on my\nYou used to, you us..."
...,...,...,...,...,...,...
245,Press,Cardi B,,46,2019,"Monsta's gon' tear it up\nBardi\nWoo, yeah\nBi..."
246,Backin' It Up,Pardison Fontaine,,47,2019,"Ahhh\nCardi!\nTurn around, fuck it all the way..."
247,Twerk,City Girls,,48,2019,"(I want a slim, fine woman with some twerk wit..."
248,Time,NF,,49,2019,Even if we both break down tonight\nAnd you sa...
